In [40]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import matplotlib.pyplot as plt


In [41]:
import cv2
import imghdr

In [42]:
data_dir = 'data/train_data/' 

In [43]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [44]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
           # os.remove(image_path)

In [45]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
print(device)

cpu


Load Data

In [47]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [48]:
train_path = 'data/train_data/'
test_path = 'data/test_data/'

In [61]:
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=128, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=16, shuffle=True
)

In [50]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])


In [51]:
print(classes)

['KanyeWest', 'TaylorSwift']


In [62]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
        #print(output)
        return output

In [53]:
model=ConvNet(num_classes=2).to(device)

In [63]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.0001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [55]:
num_epochs=30

In [56]:
# train_count=len(glob.glob(train_path+'/**/*.jpg'))
# test_count=len(glob.glob(test_path+'/**/*.jpg'))
train_count=len(glob.glob(train_path+'/**/*'))
test_count=len(glob.glob(test_path+'/**/*'))

In [57]:
print(train_count,test_count)

912 107


In [58]:
def softmax(values):
 
    # Computing element wise exponential value
    exp_values = np.exp(values)
 
    # Computing sum of these values
    exp_values_sum = np.sum(exp_values)
 
    # Returing the softmax output.
    return exp_values/exp_values_sum

In [64]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        #print(outputs)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        # print(i,labels.data, images.shape)
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        # print(np.max(outputs.data.numpy(),1))
        _,prediction=torch.max(outputs.data,1)
        values = np.apply_along_axis(softmax, 1, outputs.data.numpy())
        #print(values)
        #print(values[prediction!=labels.data],prediction != labels.data)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        # print(prediction, labels.data)
        for j in range(len(prediction)):
            # print(f"Label: {labels.data} Prediction: {prediction} Values: {values}")
            if prediction[j] != labels.data[j]:
                # plt.imshow(images[j].cpu().numpy().transpose(1,2,0))
                # plt.show()
                print(f"Label: {classes[labels.data[j]]} Prediction: {classes[prediction[j]]} Values: {values[j]}")
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        #torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Label: TaylorSwift Prediction: KanyeWest Values: [0.68749243 0.3125075 ]
Label: KanyeWest Prediction: TaylorSwift Values: [1.7375287e-06 9.9999821e-01]
Label: TaylorSwift Prediction: KanyeWest Values: [0.95203507 0.04796489]
Label: TaylorSwift Prediction: KanyeWest Values: [0.55763865 0.4423613 ]
Label: KanyeWest Prediction: TaylorSwift Values: [0.00154972 0.9984503 ]
Label: TaylorSwift Prediction: KanyeWest Values: [0.8692002  0.13079979]
Label: KanyeWest Prediction: TaylorSwift Values: [5.6463305e-09 1.0000000e+00]
Label: KanyeWest Prediction: TaylorSwift Values: [0.01570687 0.98429304]
Label: KanyeWest Prediction: TaylorSwift Values: [1.4877606e-04 9.9985123e-01]
Label: TaylorSwift Prediction: KanyeWest Values: [9.9919367e-01 8.0632989e-04]
Label: KanyeWest Prediction: TaylorSwift Values: [1.5653002e-05 9.9998438e-01]
Epoch: 0 Train Loss: tensor(0.0776) Train Accuracy: 0.9879385964912281 Test Accuracy: 0.897196261682243
Label: TaylorSwift Prediction: KanyeWest Values: [1.000000e+00 

In [ ]:
# plt.imshow(img.permute(1,2,0))
#    plt.show()
#    print(f"Label: {label}")